In [ ]:
pip install numpy pandas tensorflow keras

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
# Cargar desde Google Drive
df_subset = pd.read_csv('/content/drive/My Drive/NLP/datos_procesados.csv')

 Creamos etiquetas binarias basadas en los ratings, dividimos los datos en características y etiquetas, y separamos esos datos en conjuntos de entrenamiento y prueba, asegurando un equilibrio entre las clases.

In [40]:
# Crear la columna de etiquetas basada en el rating
df_subset['label'] = (df_subset['rating'] >= 4).astype(int)

# Dividir los datos en características (X) y etiquetas (y)
X = df_subset['processed_text']
y = df_subset['label']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Tamaño del conjunto de entrenamiento: {len(X_train)}")
print(f"Tamaño del conjunto de prueba: {len(X_test)}")
print(f"Distribución de clases en el conjunto de entrenamiento:\n{y_train.value_counts(normalize=True)}")
print(f"Distribución de clases en el conjunto de prueba:\n{y_test.value_counts(normalize=True)}")

Tamaño del conjunto de entrenamiento: 6391
Tamaño del conjunto de prueba: 1598
Distribución de clases en el conjunto de entrenamiento:
label
0    0.500235
1    0.499765
Name: proportion, dtype: float64
Distribución de clases en el conjunto de prueba:
label
0    0.5
1    0.5
Name: proportion, dtype: float64


In [42]:
# Calcular la longitud de cada texto
df_subset['text_length'] = df_subset['processed_text'].apply(lambda x: len(x.split()))

# Calcular longitud promedio y máxima
average_length = df_subset['text_length'].mean()
max_length = df_subset['text_length'].max()

print(f"Longitud promedio de los textos: {average_length}")
print(f"Longitud máxima de los textos: {max_length}")

Longitud promedio de los textos: 66.37726874452372
Longitud máxima de los textos: 1562


Tokenización y Preprocesamiento del Texto

In [43]:
max_words = 8000  # Número máximo de palabras a considerar
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

# Convertir texto a secuencias
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Rellenar secuencias para que tengan la misma longitud
max_len = 150  # Longitud máxima de las secuencias
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

Construcción del modelo LSTM y evaluación.

In [44]:
# Definir el modelo
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, return_sequences=True))  # Primera capa LSTM
model.add(Dropout(0.5))  # Regularización
model.add(LSTM(64))  # Segunda capa LSTM
model.add(Dropout(0.5))  # Regularización adicional
model.add(Dense(1, activation='sigmoid'))  # Salida para clasificación binaria

print(model.summary())

# Compilar el modelo con una tasa de aprendizaje ajustada
learning_rate = 0.001
optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Configurar Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_14 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [45]:
# Entrenar el modelo con un tamaño de lote ajustado
batch_size = 64
history = model.fit(X_train_pad, y_train, epochs=20, batch_size=batch_size,
                    validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 63s 731ms/step - accuracy: 0.5858 - loss: 0.6599 - val_accuracy: 0.7740 - val_loss: 0.5256
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 78s 689ms/step - accuracy: 0.8881 - loss: 0.2978 - val_accuracy: 0.8045 - val_loss: 0.4420
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 58s 721ms/step - accuracy: 0.9408 - loss: 0.1753 - val_accuracy: 0.8084 - val_loss: 0.5529
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 57s 711ms/step - accuracy: 0.9608 - loss: 0.1106 - val_accuracy: 0.8100 - val_loss: 0.6437
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 82s 704ms/step - accuracy: 0.9715 - loss: 0.0849 - val_accuracy: 0.7952 - val_loss: 0.7953


In [46]:
# Evaluación del Modelo
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Accuracy: {accuracy:.2f}')

50/50 ━━━━━━━━━━━━━━━━━━━━ 6s 114ms/step - accuracy: 0.8305 - loss: 0.3972
Accuracy: 0.83


In [47]:
# Predicciones y Análisis
predictions = model.predict(X_test_pad)
predicted_classes = (predictions > 0.5).astype(int)  # Convertir probabilidades a clases

from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(y_test, predicted_classes))
print(confusion_matrix(y_test, predicted_classes))

50/50 ━━━━━━━━━━━━━━━━━━━━ 7s 118ms/step
              precision    recall  f1-score   support

           0       0.83      0.83      0.83       799
           1       0.83      0.83      0.83       799

    accuracy                           0.83      1598
   macro avg       0.83      0.83      0.83      1598
weighted avg       0.83      0.83      0.83      1598

[[660 139]
 [138 661]]


El modelo de clasificación binaria utilizando LSTM ha logrado un accuracy del 83%, con métricas equilibradas entre las dos clases: una precisión de 0.83 y un recall de 0.83 para la clase 0, y una precisión de 0.83 y un recall de 0.83 para la clase 1. La matriz de confusión muestra que el modelo clasifica correctamente la mayoría de los ejemplos, aunque presenta algunos falsos positivos y negativos. A lo largo del entrenamiento, se observó una mejora rápida en las primeras épocas, pero también un ligero sobreajuste a partir de la tercera época. Para mejorar el rendimiento, se puede aumentar la regularización, ajustar hiperparámetros, implementar técnicas de aumento de datos y realizar un análisis de errores para identificar patrones que el modelo podría estar pasando por alto. En general, el modelo muestra un buen equilibrio entre precisión y recall, lo que sugiere una clasificación efectiva en este conjunto de datos.